In [102]:
import pandas as pd
import numpy as np
import os
import re

# Preprocessing Header

In [76]:
df = pd.read_csv("../Raw Data/extracted_header.csv")

In [77]:
df.head(n=3)

,sender_email,reply_to_email,subject,inline_image_count,Label
0,sh@insiq.us,sh@insiq.us,uaranteed for Eight Years,5,Spam
1,greatoffers@sendgreatoffers.com,-COM@SMTP1.ADMANMAIL.COM,ratulations! You Get a Free Handheld Organizer!,6,Spam
2,viagra_medication1182@martyrs.com.au,viagra_medication1182@martyrs.com.au,ne Doctors will fill your Viagra Prescription ...,0,Spam


In [78]:
df.isna().sum()

sender_email           0
reply_to_email         0
subject               11
inline_image_count     0
Label                  0
dtype: int64

In [79]:
df.shape

(12484, 5)

In [80]:
df = df.dropna()

In [81]:
df.Label.value_counts() 

Spam    10074
Ham      2399
Name: Label, dtype: int64

In [82]:
df.groupby("Label")["inline_image_count"].agg(["mean","median"])

,mean,median
Label,,
Ham,0.000000,0
Spam,0.623784,0


In [83]:
df[df.Label == "Ham"]["inline_image_count"].unique() # All Ham emails don't have a single inline image

array([0])

In [84]:
a=df[df.Label == "Spam"]["inline_image_count"]

In [85]:
(a == 0).sum() / len(a) * 100 # 86% of Spam Emails don't have a single inline image

85.4377605717689

In [86]:
df["has_inline_image"] = df["inline_image_count"] >= 1
df.drop("inline_image_count",axis=1,inplace=True)

In [87]:
df[df.Label == "Ham"][["sender_email","reply_to_email"]].sample(10)

,sender_email,reply_to_email
10155,liblit@eecs.berkeley.edu,rpm-zzzlist-admin@freshrpms.net
10856,gkm@petting-zoo.net,0xdeadbeef-request@petting-zoo.net
11910,tjb@unh.edu,rpm-zzzlist-admin@freshrpms.net
10334,dbr@greenhydrant.com,razor-users-admin@example.sourceforge.net
11128,cwg-exmh@DeepEddy.Com,exmh-workers-admin@example.com
10591,ygingras@ygingras.net,example.com@securityfocus.com
11627,beberg@mithral.com,fork-admin@xent.com
10565,kre@munnari.OZ.AU,exmh-workers-admin@example.com
11493,neugens@libero.it,neugens@libero.it
11349,wkearney99@hotmail.com,fork-admin@xent.com


In [88]:
df[df.Label == "Spam"][["sender_email","reply_to_email"]].sample(10)

,sender_email,reply_to_email
5867,Winston@atlaswebmail.c,Winston@atlaswebmail.com
4250,MATTHEWBUCHANAN@terra.e,None
8027,drcheick@tsamail.co.z,None
6073,stnxrk@worldsite.n,None
19,jaqwill@msn.com,smave@datapart.no
3604,allenrobert1@uno.e,None
5785,n.gleason_gu@stanghal.co.u,None
2874,kdlwtdp@broadcast.n,None
332,hezf855@framesetup.com,hezf855@framesetup.com
7567,m_ndeye2@voila.f,None


In [92]:
df["has_reply_to_email"] = df["reply_to_email"].map(lambda x: x=="None")
df.drop("reply_to_email",axis=1,inplace=True)
df.groupby("Label")["has_reply_to_email"].agg(["mean"])*100

,mean
Label,
Ham,0.000000
Spam,71.808616


In [93]:
df.head()

,sender_email,subject,Label,has_inline_image,has_reply_to_email
0,sh@insiq.us,uaranteed for Eight Years,Spam,True,False
1,greatoffers@sendgreatoffers.com,ratulations! You Get a Free Handheld Organizer!,Spam,True,False
2,viagra_medication1182@martyrs.com.au,ne Doctors will fill your Viagra Prescription ...,Spam,False,False
3,vbi@insiq.us,Sale - Three Commission Streams,Spam,True,False
4,newpsychic414@mail.ru,"Peace, Harmony, Tranquility, And Happiness Ri...",Spam,True,False


In [107]:
tld_pattern_compiled = re.compile(r"(@.+\.)(\w+)")
def get_tld(email):
    return tld_pattern_compiled.search(email).groups()[1]
    
df["top_level_domain"] = df["sender_email"].apply(get_tld)

In [114]:
df["top_level_domain"].value_counts()

c            5489
n            1758
com          1669
net           498
d             316
             ... 
nl              1
specified       1
jp              1
Net             1
ro              1
Name: top_level_domain, Length: 79, dtype: int64